In [5]:
import pandas as pd
import numpy as np
import random

In [2]:
file_path = '../bar_movement/data/'
currency_pair = 'Eur_Usd'

In [3]:
def squeeze(barsdata, length=20, length_kc=20, bb_mult=2.0, kc_mult_high=1.0, kc_mult_mid=1.5, kc_mult_low=2.0):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    bb_upper = m_avg + bb_mult * m_std
    bb_lower = m_avg - bb_mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    kc_upper_high = m_avg + range_ma * kc_mult_high
    kc_lower_high = m_avg - range_ma * kc_mult_high
    kc_upper_mid = m_avg + range_ma * kc_mult_mid
    kc_lower_mid = m_avg - range_ma * kc_mult_mid
    kc_upper_low = m_avg + range_ma * kc_mult_low
    kc_lower_low = m_avg - range_ma * kc_mult_low

    # Squeeze
    low_squeeze = (bb_lower >= kc_lower_low) | (bb_upper <= kc_upper_low) # Black
    mid_squeeze = (bb_lower >= kc_lower_mid) | (bb_upper <= kc_upper_mid) # Yellow
    high_squeeze = (bb_lower >= kc_lower_high) | (bb_upper <= kc_upper_high) # Red

    squeeze_values = np.where(high_squeeze, 'red', np.where(mid_squeeze, 'yellow', np.where(low_squeeze, 'black', 'green')))

    # Momentum
    highest_high = barsdata['Mid_High'].rolling(window=length).max()
    lowest_low = barsdata['Mid_Low'].rolling(window=length).min()
    avg_high_low = (highest_high + lowest_low) / 2
    avg_avg_high_low_sma = (avg_high_low + m_avg) / 2
    diff = barsdata['Mid_Close'] - avg_avg_high_low_sma
    squeeze_momentum = diff.rolling(window=length).apply(lambda x: np.polyfit(np.arange(length), x, 1)[0] * (length - 1) + np.polyfit(np.arange(length), x, 1)[1], raw=True)
    iff_1 = np.where(squeeze_momentum > squeeze_momentum.shift(), 'aqua', 'blue')
    iff_2 = np.where(squeeze_momentum < squeeze_momentum.shift(), 'red', 'yellow')
    squeeze_momentum_color = np.where(squeeze_momentum > 0, iff_1, iff_2)

    return squeeze_values, squeeze_momentum_color

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def smma(closes, length):
    smma = []

    for i in range(len(closes)):
        if i < length:
            smma.append(closes.iloc[:i + 1,].rolling(length).mean().iloc[-1,])

        else:
            smma.append((smma[i - 1] * (length - 1) + closes[i]) / length)

    return pd.Series(smma)

In [4]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]
rounding = 3 if 'Jpy' in currency_pair else 5

In [5]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier, use_majority_momemntum, pips_to_risk):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    trade = None
    bullish_momentum_colors, bearish_momentum_colors = {'aqua', 'blue'}, {'red', 'yellow'}

    df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_2022-2023.csv')
    df.Date = pd.to_datetime(df.Date, utc=True)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], atr_multiplier=atr_band_multiplier)
    df['squeeze_value'], df['squeeze_momentum_color'] = squeeze(df)
    df['smma200'] = smma(df['Mid_Close'], 200)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    def _recent_squeeze(curr_idx):
        max_lookback, start_idx = curr_idx - 2 - lookback, curr_idx - 2
        num_reds_seen, end_of_green_idx = 0, start_idx
        num_bearish_momentum, num_bullish_momentum = 0, 0

        for j in range(start_idx, max_lookback, -1):
            curr_squeeze_value, curr_momemntum_color = df.loc[df.index[j], ['squeeze_value', 'squeeze_momentum_color']]

            if curr_squeeze_value == 'green':
                end_of_green_idx = j
                num_bearish_momentum += 1 if curr_momemntum_color in bearish_momentum_colors else 0
                num_bullish_momentum += 1 if curr_momemntum_color in bullish_momentum_colors else 0

            else:
                break

        for j in range(end_of_green_idx - 1, max_lookback, -1):
            curr_squeeze_value = df.loc[df.index[j], 'squeeze_value']
            num_reds_seen += 1 if curr_squeeze_value == 'red' else 0

            if curr_squeeze_value == 'green':
                return False, 0, 0

            if num_reds_seen >= n_red:
                return True, num_bearish_momentum, num_bullish_momentum

        return False, 0, 0

    for i in range(lookback + 2, len(df)):
        curr_date, curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Date', 'Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]

        if trade is None:
            spread = abs(curr_ao - curr_bo)
            squeeze_value2 = df.loc[df.index[i - 2], 'squeeze_value']
            squeeze_value1, squeeze_momentum_color1, mid_close1, smma1, lower_atr_band1, upper_atr_band1 = df.loc[df.index[i - 1], ['squeeze_value', 'squeeze_momentum_color', 'Mid_Close', 'smma200', 'lower_atr_band', 'upper_atr_band']]

            if squeeze_value2 == 'green' and squeeze_value1 == 'black':
                squeeze_recently_on, num_bearish_momentum, num_bullish_momentum = _recent_squeeze(i)

                if squeeze_recently_on:
                    if use_majority_momemntum:
                        buy_signal = num_bearish_momentum > num_bullish_momentum
                        sell_signal = num_bullish_momentum > num_bearish_momentum

                    else:
                        buy_signal = squeeze_momentum_color1 in bearish_momentum_colors
                        sell_signal = squeeze_momentum_color1 in bullish_momentum_colors

                    if use_smma:
                        buy_signal = buy_signal and mid_close1 > smma1
                        sell_signal = sell_signal and mid_close1 < smma1

                    if invert:
                        buy_signal, sell_signal = sell_signal, buy_signal

                    if buy_signal:
                        open_price = float(curr_ao)
                        stop_loss = round(lower_atr_band1, rounding) if pips_to_risk is None else round(open_price - pips_to_risk, rounding)

                        if stop_loss < open_price:
                            curr_pips_to_risk = open_price - stop_loss

                            if spread <= curr_pips_to_risk * 0.1:
                                n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                                trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                        'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                        'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                                
                                pips_risked.append(curr_pips_to_risk)
                                n_buys += 1

                    elif sell_signal:
                        open_price = float(curr_bo)
                        stop_loss = round(upper_atr_band1, rounding) if pips_to_risk is None else round(open_price + pips_to_risk, rounding)

                        if stop_loss > open_price:
                            curr_pips_to_risk = stop_loss - open_price

                            if spread <= curr_pips_to_risk * 0.1:
                                n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                                trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                        'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                        'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                                
                                pips_risked.append(curr_pips_to_risk)
                                n_sells += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and not use_traditional_tsl and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and use_traditional_tsl and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
            trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and not use_traditional_tsl and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and use_traditional_tsl and trade['trade_type'] == 'sell' and curr_ask_low + trade['pips_risked'] < trade['stop_loss']:
            trade['stop_loss'] = curr_ask_low + trade['pips_risked']

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts


In [7]:
# n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier
n_reds = [3, 5, 7]
lookbacks = [50, 100]
invert_vals = [True, False]
use_traditional_tsl_vals = [True, False]
use_smma_vals = [True, False]
atr_band_multipliers = [3, 5, 7, 9]
use_majority_momemntum_vals = [True, False]
pips_to_risk_vals = [0.0020, 0.0030, 0.0040]

all_combos = []

for n_red in n_reds:
    for lookback in lookbacks:
        for invert in invert_vals:
            for use_traditional_tsl in use_traditional_tsl_vals:
                for use_smma in use_smma_vals:
                    for atr_band_multiplier in atr_band_multipliers:
                        for use_majority_momemntum in use_majority_momemntum_vals:
                            for pips_to_risk in pips_to_risk_vals:
                                all_combos.append((n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier, use_majority_momemntum, pips_to_risk))

best_n_red, best_lookback, best_invert, best_use_traditional_tsl, best_use_smma, best_atr_band_multiplier, best_use_majority_momemntum, best_pips_to_risk = None, None, None, None, None, None, None, None
top_n_results, best_rewards, best_reward, runs_finished = 10, [], -np.inf, 0

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier, use_majority_momemntum, pips_to_risk in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts = run_simulation(n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier, use_majority_momemntum, pips_to_risk)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'n_red': n_red, 'lookback': lookback, 'invert': invert, 'use_traditional_tsl': use_traditional_tsl, 'use_smma': use_smma, 'atr_band_multiplier': atr_band_multiplier, 'use_majority_momemntum': use_majority_momemntum, 'pips_to_risk': pips_to_risk})

    if total_profit > best_reward:
        best_reward = total_profit
        best_n_red, best_lookback, best_invert, best_use_traditional_tsl, best_use_smma, best_atr_band_multiplier, best_use_majority_momemntum, best_pips_to_risk = n_red, lookback, invert, use_traditional_tsl, use_smma, atr_band_multiplier, use_majority_momemntum, pips_to_risk
 
    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1152

0.0 0.0 0.0
Num buys: 1
Num sells: 1
Num trades: 2
Num wins: 1
Num losses: 1
Win streak: 1
Loss streak: 1
Avg pips risked: 0.0040000000000000036
Avg win amount: 49.996000000000045
Min win amount: 49.996000000000045
Max win amount: 49.996000000000045
Avg loss amount: -49.996000000000045
Min loss amount: -49.996000000000045
Max loss amount: -49.996000000000045
Remaining runs: 1151
Best reward so far: 0.0

-135.73913999999974 -2.4998 -138.23893999999973
Num buys: 5
Num sells: 4
Num trades: 9
Num wins: 3
Num losses: 5
Win streak: 2
Loss streak: 4
Avg pips risked: 0.0040000000000000036
Avg win amount: 19.665093333333573
Min win amount: 5.249580000001087
Max win amount: 41.746659999998734
Avg loss amount: -38.9468840000001
Min loss amount: -49.871009999999224
Max loss amount: -19.498440000000073
Remaining runs: 1150
Best reward so far: 0.0

-149.99400000000014 -1.6666 -151.66060000000013
Num buys: 2
Num sells: 2
Num trades: 4
Num wins: 0
Num losses: 3
Win streak: 1
Loss strea

In [7]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best n red: ' + str(best_n_red))
print('Best lookback: ' + str(best_lookback))
print('Best invert val: ' + str(best_invert))
print('Best use traditional tsl val: ' + str(best_use_traditional_tsl))
print('Best use smma val ' + str(best_use_smma))
print('Best atr band multiplier: ' + str(best_atr_band_multiplier))
print('Best use majority momentum: ' + str(best_use_majority_momemntum))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 535.6184799999995
Best n red: 3
Best lookback: 50
Best invert val: True
Best use traditional tsl val: False
Best use smma val False
Best atr band multiplier: 7
Best use majority momentum: False
-----------------------
Top results:
{'reward': 301, 'n_red': 7, 'lookback': 100, 'invert': False, 'use_traditional_tsl': True, 'use_smma': False, 'atr_band_multiplier': 3, 'use_majority_momemntum': False}
{'reward': 406, 'n_red': 5, 'lookback': 100, 'invert': False, 'use_traditional_tsl': True, 'use_smma': False, 'atr_band_multiplier': 3, 'use_majority_momemntum': True}
{'reward': 348, 'n_red': 5, 'lookback': 100, 'invert': False, 'use_traditional_tsl': True, 'use_smma': False, 'atr_band_multiplier': 5, 'use_majority_momemntum': False}
{'reward': 481, 'n_red': 3, 'lookback': 100, 'invert': True, 'use_traditional_tsl': False, 'use_smma': False, 'atr_band_multiplier': 7, 'use_majority_momemntum': False}
{'reward': 344, 'n_red': 5, 'lookback': 5